# TP : Systèmes de Recommandation Hybride

#  Etape 1 : Chargement des données MovieLens

In [9]:
pip install pandas scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\siham\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd

# Chargement des évaluations
df_ratings = pd.read_csv('C:/Users/siham/Desktop/projet/ml-100k/ml-100k/u.data', sep='\t', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])

# Chargement des informations sur les films
movies_df = pd.read_csv('C:/Users/siham/Desktop/projet/ml-100k/ml-100k/u.item', sep='|', header=None, encoding='latin-1', names=['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url'] + [f'genre_{i}' for i in range(19)])

In [3]:
df_ratings.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
# Préparation des DataFrames
df_ratings = df_ratings[['user_id', 'item_id', 'rating']]
movies_df = movies_df[['movie_id', 'title'] + [col for col in movies_df.columns if 'genre_' in col]]

In [5]:
movies_df

,movie_id,title,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,...,genre_9,genre_10,genre_11,genre_12,genre_13,genre_14,genre_15,genre_16,genre_17,genre_18
0,1,Toy Story (1995),0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
merged_df = df_ratings.merge(movies_df, left_on='item_id', right_on='movie_id')

In [7]:
# Construire le profil utilisateur basé sur les genres préférés
user_profiles = merged_df.groupby('user_id').apply(lambda x: pd.Series(x[[col for col in x.columns if 'genre_' in col]].mean())).reset_index()
user_profiles.columns = ['user_id'] + [f'avg_genre_{i}' for i in range(19)]

print(user_profiles.head())

   user_id  avg_genre_0  avg_genre_1  avg_genre_2  avg_genre_3  avg_genre_4  \
0        1     0.003676     0.275735     0.154412     0.044118     0.091912   
1        2     0.000000     0.161290     0.048387     0.016129     0.064516   
2        3     0.000000     0.259259     0.074074     0.000000     0.000000   
3        4     0.000000     0.333333     0.166667     0.000000     0.000000   
4        5     0.005714     0.320000     0.188571     0.080000     0.165714   

   avg_genre_5  avg_genre_6  avg_genre_7  avg_genre_8  avg_genre_9  \
0     0.334559     0.091912     0.018382     0.393382     0.007353   
1     0.258065     0.145161     0.000000     0.564516     0.016129   
2     0.222222     0.185185     0.018519     0.407407     0.000000   
3     0.166667     0.166667     0.041667     0.250000     0.000000   
4     0.468571     0.051429     0.000000     0.154286     0.011429   

   avg_genre_10  avg_genre_11  avg_genre_12  avg_genre_13  avg_genre_14  \
0      0.003676      0.047794

# Etape 2  : Filtrage collaboratif (Collaborative Filtering)


In [11]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import numpy as np

# Chargement des données
df_ratings = pd.read_csv('C:/Users/siham/Desktop/projet/ml-100k/ml-100k/u.data', sep='\t', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])


# Création de la matrice utilisateur-item
user_movie_ratings = df_ratings.pivot(index='user_id', columns='item_id', values='rating').fillna(0)

# Calcul de la similarité cosinus entre utilisateurs
user_similarity = cosine_similarity(user_movie_ratings)
user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_ratings.index, columns=user_movie_ratings.index)

print(user_similarity_df.head()) 

user_id       1         2         3         4         5         6         7    \
user_id                                                                         
1        1.000000  0.166931  0.047460  0.064358  0.378475  0.430239  0.440367   
2        0.166931  1.000000  0.110591  0.178121  0.072979  0.245843  0.107328   
3        0.047460  0.110591  1.000000  0.344151  0.021245  0.072415  0.066137   
4        0.064358  0.178121  0.344151  1.000000  0.031804  0.068044  0.091230   
5        0.378475  0.072979  0.021245  0.031804  1.000000  0.237286  0.373600   

user_id       8         9         10   ...       934       935       936  \
user_id                                ...                                 
1        0.319072  0.078138  0.376544  ...  0.369527  0.119482  0.274876   
2        0.103344  0.161048  0.159862  ...  0.156986  0.307942  0.358789   
3        0.083060  0.061040  0.065151  ...  0.031875  0.042753  0.163829   
4        0.188060  0.101284  0.060859  ...  0.052107

In [12]:
# Diviser les données en ensemble d'entraînement et de test
train_data, test_data = train_test_split(df_ratings, test_size=0.25, random_state=42)

# Recréer la matrice utilisateur-item pour l'ensemble d'entraînement
train_user_movie_ratings = train_data.pivot(index='user_id', columns='item_id', values='rating').fillna(0)
# Exemple de recommandation pour un utilisateur
user_id = 1  # Choisir un utilisateur
print(train_user_movie_ratings)

item_id  1     2     3     4     5     6     7     8     9     10    ...  \
user_id                                                              ...   
1         0.0   3.0   4.0   0.0   3.0   0.0   4.0   0.0   5.0   0.0  ...   
2         4.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
3         0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
4         0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
5         4.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
...       ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
939       0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   5.0   0.0  ...   
940       0.0   0.0   0.0   2.0   0.0   0.0   0.0   0.0   3.0   0.0  ...   
941       5.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
942       0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
943       0.0   5.0   0.0   0.0   0.0   0.0   0.0   0.0   3.0   0.0  ...   

item_id  16

# Etape 3 : Filtrage basé sur le contenu (Content-Based Filtering)

In [37]:
import pandas as pd

# Charger le fichier u.item
movies_df = pd.read_csv('C:/Users/siham/Desktop/projet/ml-100k/ml-100k/u.item', 
                         sep='|', 
                         header=None, 
                         encoding='latin-1')

# Ajouter des noms de colonnes
movies_df.columns = ['movie_id', 'title', 'release_date', 'video_release_date', 
                     'imdb_url'] + [f'genre_{i}' for i in range(19)]

# Afficher les premières lignes pour vérifier le chargement
print(movies_df.head())



   movie_id              title release_date  video_release_date  \
0         1   Toy Story (1995)  01-Jan-1995                 NaN   
1         2   GoldenEye (1995)  01-Jan-1995                 NaN   
2         3  Four Rooms (1995)  01-Jan-1995                 NaN   
3         4  Get Shorty (1995)  01-Jan-1995                 NaN   
4         5     Copycat (1995)  01-Jan-1995                 NaN   

                                            imdb_url  genre_0  genre_1  \
0  http://us.imdb.com/M/title-exact?Toy%20Story%2...        0        0   
1  http://us.imdb.com/M/title-exact?GoldenEye%20(...        0        1   
2  http://us.imdb.com/M/title-exact?Four%20Rooms%...        0        0   
3  http://us.imdb.com/M/title-exact?Get%20Shorty%...        0        1   
4  http://us.imdb.com/M/title-exact?Copycat%20(1995)        0        0   

   genre_2  genre_3  genre_4  ...  genre_9  genre_10  genre_11  genre_12  \
0        0        1        1  ...        0         0         0         0   


In [38]:
# Charger uniquement les premières lignes pour tester
movies_df = pd.read_csv('C:/Users/siham/Desktop/projet/ml-100k/ml-100k/u.item', 
                         sep='|', 
                         header=None, 
                         encoding='latin-1',
                         nrows=10)

print(movies_df)


   0                                                  1            2   3   \
0   1                                   Toy Story (1995)  01-Jan-1995 NaN   
1   2                                   GoldenEye (1995)  01-Jan-1995 NaN   
2   3                                  Four Rooms (1995)  01-Jan-1995 NaN   
3   4                                  Get Shorty (1995)  01-Jan-1995 NaN   
4   5                                     Copycat (1995)  01-Jan-1995 NaN   
5   6  Shanghai Triad (Yao a yao yao dao waipo qiao) ...  01-Jan-1995 NaN   
6   7                              Twelve Monkeys (1995)  01-Jan-1995 NaN   
7   8                                        Babe (1995)  01-Jan-1995 NaN   
8   9                            Dead Man Walking (1995)  01-Jan-1995 NaN   
9  10                                 Richard III (1995)  22-Jan-1996 NaN   

                                                  4   5   6   7   8   9   ...  \
0  http://us.imdb.com/M/title-exact?Toy%20Story%2...   0   0   0   1   

In [40]:
print(movies_df.columns)

Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23],
      dtype='int64')


In [43]:
import pandas as pd

# Charger le DataFrame en utilisant les premiers éléments pour tester
movies_df = pd.read_csv('C:/Users/siham/Desktop/projet/ml-100k/ml-100k/u.item', 
                         sep='|', 
                         header=None, 
                         encoding='latin-1',
                         nrows=10)

# Vérifiez les colonnes de votre DataFrame
print(movies_df.columns)

# Définir les indices des colonnes de genre (à partir de la colonne 5 à la colonne 23)
genre_columns = list(range(5, 24))  # Cela couvre les colonnes de genre 0 à 18

# Liste des noms de genres
genre_names = [
    "Action", "Adventure", "Animation", "Children's", "Comedy", "Crime",
    "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical",
    "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"
]

# Créer la colonne genres
movies_df['genres'] = movies_df[genre_columns].apply(
    lambda x: ' '.join([genre_names[i] for i, value in enumerate(x) if value == 1]), 
    axis=1
)

# Vérifiez maintenant le contenu de la colonne genres
print(movies_df['genres'].unique())


Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23],
      dtype='int64')
["Children's Comedy Crime" 'Adventure Animation War' 'War'
 'Adventure Crime Fantasy' 'Documentary Fantasy War' 'Fantasy'
 'Fantasy Thriller' 'Comedy Crime Fantasy' 'Fantasy Western']


In [49]:
import pandas as pd

# Charger le DataFrame en utilisant les premiers éléments pour tester
movies_df = pd.read_csv('C:/Users/siham/Desktop/projet/ml-100k/ml-100k/u.item', 
                         sep='|', 
                         header=None, 
                         encoding='latin-1')

# Afficher les premières lignes et la structure du DataFrame
print(movies_df.head())
print("Nombre de colonnes dans le DataFrame : ", movies_df.shape[1])

# Définir les indices des colonnes de genre
genre_columns = list(range(5, 24))  # De la colonne 5 à la colonne 23
print("Indices des colonnes de genre : ", genre_columns)

# Liste des noms de genres (corriger pour avoir 19 genres)
genre_names = [
    "Action", "Adventure", "Animation", "Children's", "Comedy", "Crime",
    "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical",
    "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western", "Unknown"
]

# Vérifiez si le nombre de genres dans le DataFrame correspond au nombre de noms de genres
if len(genre_columns) != len(genre_names):
    print(f"Nombre de genres dans les colonnes: {len(genre_columns)}")
    print(f"Nombre de noms de genres: {len(genre_names)}")
    raise ValueError("Le nombre de colonnes de genre ne correspond pas au nombre de noms de genres.")

# Créer la colonne genres
movies_df['genres'] = movies_df[genre_columns].apply(
    lambda x: ' '.join([genre_names[i] for i, value in enumerate(x) if value == 1]), 
    axis=1
)

# Vérifiez maintenant le contenu de la colonne genres
print("Exemples de genres : ", movies_df['genres'].unique())


   0                  1            2   3   \
0   1   Toy Story (1995)  01-Jan-1995 NaN   
1   2   GoldenEye (1995)  01-Jan-1995 NaN   
2   3  Four Rooms (1995)  01-Jan-1995 NaN   
3   4  Get Shorty (1995)  01-Jan-1995 NaN   
4   5     Copycat (1995)  01-Jan-1995 NaN   

                                                  4   5   6   7   8   9   ...  \
0  http://us.imdb.com/M/title-exact?Toy%20Story%2...   0   0   0   1   1  ...   
1  http://us.imdb.com/M/title-exact?GoldenEye%20(...   0   1   1   0   0  ...   
2  http://us.imdb.com/M/title-exact?Four%20Rooms%...   0   0   0   0   0  ...   
3  http://us.imdb.com/M/title-exact?Get%20Shorty%...   0   1   0   0   0  ...   
4  http://us.imdb.com/M/title-exact?Copycat%20(1995)   0   0   0   0   0  ...   

   14  15  16  17  18  19  20  21  22  23  
0   0   0   0   0   0   0   0   0   0   0  
1   0   0   0   0   0   0   0   1   0   0  
2   0   0   0   0   0   0   0   1   0   0  
3   0   0   0   0   0   0   0   0   0   0  
4   0   0   0   0   0 

# Etape 4 : Système hybride

In [14]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Extrait la matrice utilisateur-film
user_movie_ratings = df_ratings.pivot(index='user_id', columns='item_id', values='rating').fillna(0)

# Calcul de la similarité entre utilisateurs (collaboratif)
user_similarity = cosine_similarity(user_movie_ratings)

# Fonction pour recommander des films basés sur les utilisateurs similaires
def recommend_collaborative(user_id, user_similarity, user_movie_ratings, n_recommendations=10):
    # Obtenir les utilisateurs similaires
    similar_users = user_similarity[user_id].argsort()[::-1][1:11]
    
    # Obtenir les films préférés des utilisateurs similaires
    recommended_movies = []
    for similar_user in similar_users:
        movies_rated = user_movie_ratings.columns[user_movie_ratings.iloc[similar_user] > 0].tolist()
        recommended_movies.extend(movies_rated)
    
    # Limiter le nombre de films recommandés
    recommended_movies = list(set(recommended_movies))[:n_recommendations]
    
    return recommended_movies

# Exemple d'utilisation
user_id = 1  # Identifiant de l'utilisateur
recommended_movies_collaborative = recommend_collaborative(user_id, user_similarity, user_movie_ratings)
print("Films recommandés par filtrage collaboratif :", recommended_movies_collaborative)


Films recommandés par filtrage collaboratif : [1, 515, 3, 1028, 6, 7, 9, 10, 13, 14]


In [15]:
def filter_by_genre(user_genres_input, recommended_movies, movies_df, tfidf_matrix, tfidf, n_recommendations=5):
    # Préparer les genres préférés de l'utilisateur
    user_genres_string = ' '.join([genre.strip().lower() for genre in user_genres_input.split(',')])
    
    # Calculer la matrice TF-IDF pour les préférences de l'utilisateur
    user_tfidf = tfidf.transform([user_genres_string])
    
    # Filtrer la matrice TF-IDF uniquement pour les films recommandés collaborativement
    recommended_movies_indices = [movies_df.index[movies_df['movie_id'] == movie_id].tolist()[0] for movie_id in recommended_movies]
    filtered_tfidf_matrix = tfidf_matrix[recommended_movies_indices]
    
    # Calculer la similarité cosinus uniquement sur les films recommandés
    cosine_sim = cosine_similarity(user_tfidf, filtered_tfidf_matrix)
    
    # Trier les films recommandés par ordre décroissant de similarité
    similar_movies_indices = cosine_sim.argsort()[0][::-1]
    
    # Extraire les films recommandés et filtrés
    filtered_movies = []
    for idx in similar_movies_indices:
        movie_idx = recommended_movies_indices[idx]
        filtered_movies.append((movies_df.iloc[movie_idx]['title'], movies_df.iloc[movie_idx]['genres_combined']))
        if len(filtered_movies) >= n_recommendations:
            break
    
    return filtered_movies


# Etape 5 : Personnalisation

In [16]:
# Fonction pour permettre à l'utilisateur de sélectionner dynamiquement ses genres préférés
def get_user_genres():
    print("Liste des genres disponibles :")
    available_genres = ['Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 
                        'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 
                        'War', 'Western']
    
    # Affichage des genres disponibles
    for i, genre in enumerate(available_genres):
        print(f"{i+1}. {genre}")
    
    # Demande à l'utilisateur de sélectionner ses genres préférés (par numéros)
    user_input = input("Sélectionnez vos genres préférés en entrant les numéros correspondants (séparés par des virgules) : ")
    
    # Conversion de l'entrée en liste de genres
    selected_indices = [int(i.strip()) - 1 for i in user_input.split(',')]
    selected_genres = [available_genres[i] for i in selected_indices]
    
    return ', '.join(selected_genres)  # Retourne une chaîne de genres sélectionnés

# Appel de la fonction pour obtenir les genres préférés de l'utilisateur
user_genres_input = get_user_genres()
print(f"Vos genres préférés sont : {user_genres_input}")


Liste des genres disponibles :
1. Action
2. Adventure
3. Animation
4. Children's
5. Comedy
6. Crime
7. Documentary
8. Drama
9. Fantasy
10. Film-Noir
11. Horror
12. Musical
13. Mystery
14. Romance
15. Sci-Fi
16. Thriller
17. War
18. Western
Vos genres préférés sont : Film-Noir
